In [32]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [33]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung','vorh'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df.head()

/home/dbsm/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,signatur,jahr,datum,medienart,sprache,land,koerperschaft_1,koerperschaft_2,person,titel,urheber_vorlage,ort,ort_idn,umfang,bemerkung,vorh
0,Bö-GR/V/1,1870,02.01.1870,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Begründung durch Ankauf von Artikeln des Verla...,Franz Vahlen,Berlin,!040057283!,5 Exemplare,Verlagsliste,ja
1,Bö-GR/V/2,1886,XX.10.1886,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Vollmachtserteilung an den Mitarbeiter Friedri...,Franz Vahlen,Berlin W.,!040057283!,6 Exemplare,<NA>,ja
2,Bö-GR/V/3,1898,07.02.1898,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Der Mitarbeiter Friedrich Gebhardt wird Teilha...,Franz Vahlen,Berlin W.,!040057283!,4 Exemplare,<NA>,ja
3,Bö-GR/V/4,1898,05.11.1898,druck,ger,XA-DXDE,Franz Vahlen,<NA>,"Gehardt, Friedrich",Friedrich Gebhardt kauft die Verlagsbuchhandlung.,Friedrich Gebhardt,Berlin W.8,!040057283!,6 Exemplare,Handschriftliches Begleitschreiben an den Börs...,ja
4,Bö-GR/V/5,1910,01.01.1910,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Der Mitarbeiter Heinrich Heise erhält Prokura.,Franz Vahlen,Berlin W 9,!040057283!,1 Exemplar,<NA>,ja


In [60]:
def pica_schreiben(row):

    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
7100 {row.signatur}
"""

    if pd.notna(row.datum):
        d2 = row.datum.split('.')

    if d2[0] == 'XX' and (d2[1]) == '01' or (d2[1]) == '03' or (d2[1]) == '05' or (d2[1]) == '07' or (d2[1]) == '08' or (d2[1]) == '10' or (d2[1]) == '12':
        pica += f"1110 $a01.{d2[1]}.{d2[2]}$b31.{d2[1]}.{d2[2]}$4ezth\n"
    elif d2[0] == 'XX' and (d2[1]) == '04' or (d2[1]) == '06' or (d2[1]) == '09' or (d2[1]) == '11':
        pica += f"1110 $a01.{d2[1]}.{d2[2]}$b30.{d2[1]}.{d2[2]}$4ezth\n"
    elif d2[0] == 'XX' and (d2[1]) == 'XX':
        pica += f"1110 $a01.01.{d2[2]}$b31.12.{d2[2]}$4ezth\n"
    else:
        pica += f"1110 $a{row.datum}$b{row.datum}$4ezth\n"
    return pica


In [61]:
now = datetime.now()
for index, row in df.sample(100).iterrows():
    print(pica_schreiben(row))

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1833
7100 Bö-GR/V/407
1110 $a01.09.1833$b30.09.1833$4ezth

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1884
7100 Bö-GR/V/653
1110 $a01.01.1884$b01.01.1884$4ezth

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1858
7100 Bö-GR/V/64
1110 $a01.12.1858$b31.12.1858$4ezth

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1903
7100 Bö-GR/V/269
1110 $a01.10.1903$b31.10.1903$4ezth

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1827
7100 Bö-GR/V/753
1110 $a05.04.1827$b05.04.1827$4ezth



UnboundLocalError: local variable 'd2' referenced before assignment

In [62]:
with open(f"../dat/1110sample.dat", 'w') as f:
    for index, row in df.iterrows():
        f.write(pica_schreiben(row))

UnboundLocalError: local variable 'd2' referenced before assignment